In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40 kB 1.7 MB/s eta 0:00:011
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-py3-none-any.whl size=30114 sha256=8add6b6b4598d4c300f4814767ee1440b896ccacbd5ae05a428af27aa710a1a0
  Stored in directory: /home/nobu_yamaguchi/.cache/pip/wheels/6c/c9/9c/363182ea34a736dae336eeaf0dd4a7eec3c6a5afe32373e1fe
  Created wheel for py-params: filename=py_params-0.9.7-py3-none-any.whl size=7302 sha256=8ac0dcbc968ce8599110cba6d6bb23899f4465fb7eea4584186b35556c449a0a
  Stored in directory: /home/nobu_yamaguchi/.cache/pip/wheels/47/3d/2d/bbffcfd6b9f4b8b5cbf07e7520ac2676192fe9431240c13ee8
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=db82dd4d94cafc56d4ee6e075c4dc30fc1c29e662d0b70567d39fb5248e18166
  Stored in directory: /home/nobu_yamaguchi/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
Successfully built bert-for-tf2 py-params params-flow
You should consider upgrading via 

In [4]:
!pip install tensorflow_hub

     |████████████████████████████████| 101 kB 2.7 MB/s ta 0:00:011
You should consider upgrading via the '/home/nobu_yamaguchi/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
import tensorflow as tf

import tensorflow_hub as hub
import numpy as up
import pandas as pd
from tensorflow.keras import layers
import bert

In [2]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

Using TensorFlow backend.


In [3]:
df_merge_quality = pd.read_csv('US_patent_abstract_5000_2015_with_title_1.csv')
df_merge_quality.shape

(5000, 30)

In [4]:
df = df_merge_quality[['claims_text', 'quality_rank']]

In [5]:
df.quality_rank.value_counts()

0    2896
1    2104
Name: quality_rank, dtype: int64

In [6]:
2896/5000

0.5792

In [10]:
df.shape

(5000, 2)

In [6]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [7]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [8]:
claims = []
sentences = list(df['claims_text'])
for sen in sentences:
    claims.append(preprocess_text(str(sen)))

In [9]:
print(df.columns.values)

['claims_text' 'quality_rank']


In [10]:
df.quality_rank.unique()

array([0, 1])

In [11]:
y = df.quality_rank

In [12]:
y[10]

1

In [13]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [14]:
def tokenize_claims(text_claims):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_claims))

In [15]:
tokenized_claims = [tokenize_claims(claim) for claim in claims]

Prerparing Data For Training

In [16]:
claims_with_len = [[claim, y[i], len(claim)]
                 for i, claim in enumerate(tokenized_claims)]

In [22]:
#np.random.shuffle(claims_with_len)

In [28]:
#claims_with_len.sort(key=lambda x: x[2])

In [17]:
sorted_claims_labels = [(claim_lab[0], claim_lab[1]) for claim_lab in claims_with_len]

In [18]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_claims_labels, output_types=(tf.int32, tf.int32))

In [19]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [20]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 4552), dtype=int32, numpy=
 array([[ 5080,  9605,  3638, ...,     0,     0,     0],
        [ 1996, 11028,  3555, ...,     0,     0,     0],
        [ 2054,  2003,  3555, ...,     0,     0,     0],
        ...,
        [ 2019,  1999,  4946, ...,     0,     0,     0],
        [ 1996, 11028,  3555, ...,     0,     0,     0],
        [ 2054,  2003,  3555, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 1], dtype=int32)>)

In [21]:
TOTAL_BATCHES = np.math.ceil(len(sorted_claims_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 20
#batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [54]:
type(y)

pandas.core.series.Series

In [56]:
x_train = np.array(tokenized_claims)[:4000]
x_test = np.array(tokenized_claims)[4000:]
y_train = np.array(y)[:4000]
y_test = np.array(y)[4000:]

In [24]:
a = 0
for element in test_data:
    a += len(element[1])
a

224

In [25]:
i = 0
for element in test_data:
    i += sum(element[1])
i

<tf.Tensor: shape=(), dtype=int32, numpy=100>

In [40]:
1-48/224

0.7857142857142857

In [22]:
from keras.engine.topology import Layer, InputSpec
from tensorflow.python.keras.layers import Lambda

In [23]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)), name='W')
        self.b = K.variable(self.init((self.attention_dim, )), name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)), name='u')
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
 
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b)) #

        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)


        #if mask is not None:
        #    # Cast the mask to floatX to avoid float64 upcasting in theano
        #    print('mask')
        #    ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        ait = K.expand_dims(ait)
 
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=200,
                 MAX_SENT_LENGTH=30,
                 MAX_SENTS=30,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.4,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.lstm = Bidirectional(GRU(100, return_sequences=True))
        
        self.l_att = AttLayer(100)
        self.sentEncoder = Model(Input(shape=(MAX_SENT_LENGTH,), dtype='int32'), )
        
        
        self.claims_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
        self.claims_encoder = TimeDistributed(self.sentEncoder)
        
        self.l_lstm_sent = Bidirectional(GRU(30, return_sequences=True))
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dense_2 = layers.Dense(units=dnn_units, activation="relu")
        self.dense_3 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.lstm(l) 
        l_2 = self.l_att(l_1)
        l_3 = self.sentEncoder
        
        l_4 = self.claims_input
        
        l_5 = claims_encoder(l_4)
        l_6 = l_lstm_sent(l_5)
        
        concatenated = self.dense_1(concatenated)
        concatenated = self.dense_2(concatenated)
        concatenated = self.dense_3(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [26]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.4

NB_EPOCHS = 5

text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        MAX_SENT_LENGTH=30,
                        MAX_SENTS=30,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [36]:
sentence_input = Input(shape=(30,), dtype='int32')
embedded_sequences = Embedding(VOCAB_LENGTH, EMB_DIM)(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_lstm)
sentEncoder = Model(sentence_input, l_att)

tracking <tf.Variable 'att_layer_4/W:0' shape=(200, 100) dtype=float32> W
tracking <tf.Variable 'att_layer_4/b:0' shape=(100,) dtype=float32> b
tracking <tf.Variable 'att_layer_4/u:0' shape=(100, 1) dtype=float32> u


In [37]:
claims_input = Input(shape=(30, 30), dtype='int32')
claims_encoder = TimeDistributed(sentEncoder)(claims_input)
l_lstm_sent = Bidirectional(GRU(30, return_sequences=True))(claims_encoder)

In [38]:
l_att_sent = AttLayer(100)(l_lstm_sent)

tracking <tf.Variable 'att_layer_5/W:0' shape=(60, 100) dtype=float32> W
tracking <tf.Variable 'att_layer_5/b:0' shape=(100,) dtype=float32> b
tracking <tf.Variable 'att_layer_5/u:0' shape=(100, 1) dtype=float32> u


In [39]:
l_att_dense = Dense(100, activation='relu')(l_att_sent)
l_att_dense = Dense(100, activation='relu')(l_att_dense)
l_att_dense = Dense(100, activation='relu')(l_att_dense)
l_att_sent_drop = Dropout(rate=0.2)(l_att_dense)
preds = Dense(2, activation='sigmoid')(l_att_sent_drop)
model = Model(claims_input, preds)

In [40]:
model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["acc"])

In [36]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["acc"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [57]:
model.fit(x_train, y_train, validation_data=(x_test, y_test),
          epochs=8, batch_size=50)

ValueError: Error when checking input: expected input_8 to have 3 dimensions, but got array with shape (4000, 1)

In [49]:
model.fit(train_data, epochs=NB_EPOCHS, validation_data=test_data)

AttributeError: 'SkipDataset' object has no attribute 'ndim'